<a href="https://colab.research.google.com/github/ZhihanX/SocialMedia_Sentiment_Analysis/blob/main/dynamic_feature_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U hopsworks --quiet
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.3/170.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 5.3 MB/s eta 0:00:00
  Preparing metada

In [ ]:
import hopsworks
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# utilities
import re
import pickle
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/188566
Connected. Call `.close()` to terminate connection gracefully.


In [ ]:
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('https://api.apify.com/v2/datasets/7gjebUVPipE3lK3HN/items?attachment=true&clean=true&format=csv',
                      encoding=DATASET_ENCODING)
dataset = dataset['full_text']
dataset

0     Congrats, England! It was a deserved victory, ...
1     Swedish meatballs are actually based on a reci...
2                             @pewdiepie Stort grattis!
3     Today thousands of people gather in Stockholm ...
4     We are totally stunned by the amazing World Cu...
                            ...                        
95    Happy 41st birthday, Crown Princess Victoria! ...
96    Today is the day of Vasaloppet, Sweden's tradi...
97    Happy New Year from Sweden! See you next decad...
98                    3-0! Love how this game is going.
99    Good morning from the beautiful city of Stockh...
Name: full_text, Length: 100, dtype: object

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
senti_label = dataset.apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [ ]:
senti_label

0     0.7654
1     0.0000
2     0.0000
3     0.5719
4     0.9018
       ...  
95    0.6114
96    0.3987
97    0.7772
98    0.6696
99    0.9097
Name: full_text, Length: 100, dtype: float64

In [ ]:
df = pd.DataFrame(columns=['text', 'label'])

In [ ]:
df['label'] = senti_label.apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df['text'] = dataset

In [ ]:
df

,text,label
0,"Congrats, England! It was a deserved victory, ...",1
1,Swedish meatballs are actually based on a reci...,0
2,@pewdiepie Stort grattis!,0
3,Today thousands of people gather in Stockholm ...,1
4,We are totally stunned by the amazing World Cu...,1
...,...,...
95,"Happy 41st birthday, Crown Princess Victoria! ...",1
96,"Today is the day of Vasaloppet, Sweden's tradi...",1
97,Happy New Year from Sweden! See you next decad...,1
98,3-0! Love how this game is going.,1


In [ ]:
twitter_fg = fs.get_or_create_feature_group(
    name="twi_sentiment_yearly",
    version=1,
    primary_key=['text','label'],
    description="Twitter dataset that update yearly")
twitter_fg.insert(df)

Uploading Dataframe: 0.00% |          | Rows 0/100 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: twi_sentiment_yearly_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/188566/jobs/named/twi_sentiment_yearly_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x78b648764ca0>, None)